In [0]:
import pandas as pd
import string

data = pd.read_csv("data.csv", index_col=0)
data = data.groupby('class')

In [0]:
def negateWords(wordlist):
  '''
  Add the tag
  'NOT' to every word between a negation word (“not”,
  “isn’t”, “didn’t”, etc.) and the first punctuation
  mark following the negation word.
  '''
  new_list = []
  i = 0
  while i < len(wordlist):
    if wordlist[i] == "not" or "n't" in wordlist[i]:
      new_list.append(wordlist[i])
      i += 1
      while wordlist[i] not in string.punctuation:
        new_list.append("NOT_" + wordlist[i])
        i += 1
    new_list.append(wordlist[i])
    i += 1
  return new_list

In [0]:
data_neg = data.get_group('neg')
data_pos = data.get_group('pos')

sample_data = data_neg.sample(frac=0.7, random_state=5)
sample_data = sample_data.append(data_pos.sample(frac=0.7, random_state=5), ignore_index=True).reset_index(drop=True)

uni_data = sample_data.copy()
uni_data['text'] = uni_data.text.apply(lambda x: x.replace('\n', '').strip().replace('  ', ' ').split())
uni_data.text = uni_data.text.apply(negateWords)

In [4]:
from collections import Counter

def build_vocab(text, min_count=4):
  '''
  Takes in a 2D list of words and returns 
  set of words occuring min_count or more times
  '''
  wl = []
  for i in text:
    wl += i
  counts = Counter(wl)
  
  vocab = set([i if j >= min_count else '' for i,j in counts.items()])
  vocab.remove('')
  return vocab, counts

vocab, counts = build_vocab(uni_data.text)
print(f"Size of unigram vocab: {len(vocab)}")

Size of unigram vocab: 15521


Transforming texts into word: word-Counts

In [0]:
wfreqs = []
for i in uni_data.text:
  wf = Counter(i)
  wfreqs.append(wf)
d = pd.DataFrame(wfreqs).fillna(0).astype('int32')
drop_c = set(d.columns) - vocab
uni_freq_data = d.drop(drop_c, axis=1)

In [21]:
uni_freq_data.head()

,the,makers,of,spawn,have,created,something,almost,as,vacuous,this,summer's,other,comic,book,adaptation,",",batman,and,robin,.,both,films,make,mistake,adapting,for,screen,not,NOT_only,NOT_the,NOT_look,NOT_of,NOT_their,NOT_graphic,but,also,their,dialogue,empty-headed,...,mckellan,boone,anney,domingo,leila,hatami,reza,NOT_goals,u2,bettany,beacham,matilda,trunchbull,zaire,ralphie,story',toback,osnard,sheldon,lauri,ilona,nomi's,benigni+s,rylance,dussander,cuba's,caraboo,worrall,fang,quincy,thora,burnham,carolyn,nesmith,cams,corleone,ness,stephane,maxime,hauer
0,16,1,11,7,1,1,3,1,3,1,1,1,1,2,2,1,23,1,11,1,13,1,1,1,1,1,5,1,3,1,4,1,1,1,1,6,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,24,0,18,0,1,0,1,0,4,0,1,0,0,0,0,0,44,0,14,0,20,0,0,0,0,0,5,0,2,1,6,0,2,1,1,2,1,4,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,16,0,3,0,1,0,0,0,1,0,2,0,1,0,0,0,14,0,6,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,27,0,24,0,5,1,0,0,8,0,0,0,5,1,0,0,32,0,14,0,29,0,1,0,0,0,3,1,6,0,4,0,2,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,61,0,33,0,2,0,1,1,7,0,8,0,2,0,0,0,56,0,17,0,67,0,0,2,0,0,10,0,1,1,6,0,3,1,0,8,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
def ngrams(wordlist, N=2):
  '''
  Takes in a list of words and returns ngrams wordlist
  '''
  new_list = []
  for i in range(len(wordlist) - N + 1):
    new_list.append('_'.join(wordlist[i:i+N]))
  return new_list

In [0]:
def process(text, vocab):
  '''
  Takes in a Series of texts and vocabulary and returns a dataframe
  with Vocabulary as columns and 1 / 0 as values indicating
  the presence of word in the list
  '''
  text = list(text.apply(lambda x: Counter(set(x) & vocab)))
  return pd.DataFrame(text).fillna(0).astype('bool')

In [0]:
bi_data = sample_data.copy()
bi_data.text = bi_data.text.apply(lambda x: x.replace('\n', '').strip().split(' ')).apply(ngrams)
bi_vocab, bi_counts = build_vocab(bi_data.text)
bi_vocab = set([i for i,_ in bi_counts.most_common(len(vocab))])

In [0]:
uni_pres_data = process(uni_data.text, vocab)
bi_pres_data = process(bi_data.text, bi_vocab)
uni_bi_data = pd.concat([uni_pres_data, bi_pres_data], axis=1)

In [38]:
uni_pres_data.head()

,in,!,death,him,played,how,motivations,crow,costume,ostensibly,but,NOT_graphic,as,suffered,",",makers,irony,with,also,.,corporate,behold,virtual,human,NOT_be,crude,storytelling,NOT_expected,wife,when,action,longs,behind,set,triumph,problems,;,he,dead,hypnotic,...,mckellan,boone,anney,domingo,leila,reza,hatami,NOT_goals,u2,bettany,beacham,trunchbull,matilda,zaire,story',ralphie,toback,osnard,sheldon,lauri,ilona,nomi's,benigni+s,rylance,dussander,cuba's,worrall,caraboo,fang,quincy,carolyn,burnham,thora,nesmith,cams,corleone,ness,stephane,maxime,hauer
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,False,False,False,False,False,True,True,True,False,True,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,True,False,False,False,False,True,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,True,False,True,True,False,False,False,False,False,True,False,True,False,True,False,False,True,True,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [39]:
bi_pres_data.head()

,have_a,not_only,",_for",",_then",to_make,._both,._in,it_),realizes_that,",_it",sequel_.,overshadowed_by,the_inevitable,makers_of,"be_,",)_.,",_it's",not_that,know_how,really_cool,more_difficult,dialogue_and,._but,and_robin,on_fire,",_here's",sequences_are,man_),for_his,for_reasons,which_was,._he,make_the,set_on,expected_to,it_had,",_but",",_and",by_the,but_also,...,x-files_is,fits_the,`final_destination',works_well,harris_),nuclear_war,truman_burbank,"""_drunken","master_""",more_powerful,black_cauldron,babe_:,:_pig,boy_is,american_history,history_x,"hunting_""",apt_pupil,high_fidelity,"""_sweet","sweetback_""",sweet_sweetback,the_apostle,caveman's_valentine_,_the_caveman's,"destination_""",the_horned,"cauldron_,",._taran,hen_wen,the_endurance,of_flanders,the_laserman,"fail-safe_""","""_fail-safe",raging_bull,la_motta,-_*,*_-,princess_caraboo
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,True,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,True,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [24]:
import nltk
nltk.download('averaged_perceptron_tagger')
POS_data = uni_data.copy()
POS_data.text = POS_data.text.apply(nltk.pos_tag)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [35]:
# POS_data.text = POS_data.text.apply(lambda x: ['_'.join(i) for i in x])
POS_vocab, POS_counts = build_vocab(POS_data.text)
POS_pres_data = process(POS_data.text, POS_vocab)
POS_pres_data.head()

,"(though, IN)","(when, WRB)","(,, ,)","((, ()","(., .)","(reality, NN)","(created, VBN)","(these, DT)","(almost, RB)","(which, WDT)","(so, RB)","(costume, NN)","(NOT_graphic, NNP)","(granted, VBD)","(minute, NN)","(it, PRP)","(sheen, NN)","(NOT_be, NNP)","(city, NN)","(determine, VB)","(summer, NN)","(screens, NNS)","(!, .)","(something, NN)","(next, JJ)","(i, NN)","(adaptation, NN)","(white, JJ)","(about, IN)","(NOT_guess, NNP)","(plots, NNS)","(become, VB)","(they, PRP)","(make, VBP)","(problems, NNS)","(that, WDT)","(batman, NN)","(;, :)","(empty-headed, JJ)","(an, DT)",...,"(bombers, NNS)","(groeteschele, NN)","(motta, NN)","(motta, FW)","(pauline, NN)","(paradine, NN)","(boone, NN)","(anney, NN)","(domingo, NN)","(leila, NN)","(hatami, NN)","(reza, NN)","(leila, JJ)","(NOT_goals, NNP)","(bettany, NN)","(matilda, NN)","(trunchbull, NN)","(zaire, NN)","(ralphie, NN)","(story', NN)","(hip-hop, NN)","(osnard, NN)","(*, SYM)","(sheldon, NN)","(benigni+s, NN)","(skylar, JJ)","(rylance, NN)","(dussander, NN)","(worrall, NN)","(caraboo, NN)","(quincy, NN)","(amistad, JJ)","(carolyn, NN)","(burnham, NN)","(cams, NNS)","(corleone, NN)","(ness, NN)","(stephane, NN)","(hauer, NN)","(rhett, NN)"
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,True,True,False,False,True,False,True,True,False,True,False,False,True,False,False,False,False,False,False,True,True,True,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,True,True,True,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,True,True,True,True,False,False,False,True,True,True,False,False,False,False,True,False,False,False,False,False,False,True,True,False,True,False,False,True,False,False,False,False,True,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [45]:
adj_vocab = set([t if "JJ" in t[1] else '' for t in POS_vocab])
adj_vocab.remove('')
adj_pres_data = process(POS_data.text, adj_vocab)
adj_pres_data.head()

,"(unbeknownst, JJ)","(other, JJ)","(pretty, JJ)","(last, JJ)","(comic, JJ)","(typical, JJ)","(many, JJ)","(unanswered, JJ)","(hypnotic, JJ)","(virtual, JJ)","(human, JJ)","(lazy, JJ)","(dead, JJ)","(genuine, JJ)","(next, JJ)","(few, JJ)","(i'm, JJ)","(white, JJ)","(similar, JJ)","(implausible, JJ)","(difficult, JJ)","(untimely, JJ)","(cool, JJ)","(spawn's, JJ)","(crude, JJ)","(thirty, JJ)","(empty-headed, JJ)","(vacuous, JJ)","(corporate, JJ)","(incoherent, JJ)","(mary, JJ)","(foul-mouthed, JJ)","(pornographic, JJ)","(fascinating, JJ)","(gratuitous, JJ)","(bored, JJ)","(harder, JJR)","(short, JJ)","(cold, JJ)","(most, JJS)",...,"(humbert, JJ)","(arthur's, JJ)","(uncompromising, JJ)","(gallo's, JJ)","(showy, JJ)","(impersonal, JJ)","(n, JJ)","(derek, JJ)","(skinhead, JJ)","(derek's, JJ)","(mathematical, JJ)","(winslow, JJ)","(unrestrained, JJ)","(donny, JJ)","(soon-tek, JJ)","(fa, JJ)","(jar-jar, JJ)","(mid-life, JJ)","(deft, JJ)","(indistinguishable, JJ)","(belgian, JJ)","(kate, JJ)","(uk, JJ)","(gavin's, JJ)","(sweetback, JJ)","(laurie, JJ)","(fox's, JJ)","(horned, JJ)","(fflewdurr, JJ)","(craven, JJ)","(brokedown, JJ)","(bowden, JJ)","(well-defined, JJ)","(nello, JJ)","(antz, JJ)","(ulee, JJ)","(fail-safe, JJ)","(leila, JJ)","(skylar, JJ)","(amistad, JJ)"
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,True,False,True,False,True,True,True,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [49]:
top_uni_vocab = set([i for i,j in counts.most_common(len(adj_vocab))])
top_uni_pres_data = process(uni_data.text, top_uni_vocab)
top_uni_pres_data.head()

,in,!,death,him,played,how,crow,costume,but,as,",",makers,with,also,.,virtual,human,NOT_be,storytelling,wife,when,action,behind,set,problems,;,he,dead,film,spawn,NOT_only,which,often,created,that,both,:,granted,was,become,...,jedi,costumes,spiritual,sword,NOT_best,threat,critic,cauldron,suggest,radio,tony,outrageous,alice,opera,cole,franchise,superb,carter,breathtaking,soap,spectacular,prior,gradually,ellie,apes,pig,sidney,ripley,colors,flynt,norton,outstanding,hanks,bulworth,duke,damon,mulan,pitt,shrek,gattaca
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,False,False,False,True,True,True,False,True,True,True,False,False,False,False,False,False,False,True,False,False,True,True,False,True,False,True,True,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True,True,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,True,False,False,True,True,True,False,True,False,True,False,False,False,False,False,True,False,False,True,False,True,True,False,True,False,False,False,False,True,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,True,False,True,True,False,False,False,True,True,True,False,True,True,True,False,True,False,False,False,True,False,False,False,False,False,True,True,False,False,True,True,False,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Tag each word
according to whether it appeared in the first quarter, last quarter, or middle half of the document.

In [54]:
uni_pos_data = uni_data.copy()
uni_pos_data.text = uni_pos_data.text.apply(lambda x: ['_'.join([j, 'FQ']) if i/len(x) < 0.25 
                                            else '_'.join([j, 'LQ']) if i/len(x) > 0.75 else '_'.join([j, 'MH']) for i,j in enumerate(x)])
uni_pos_data.head()

,text,class
0,"[the_FQ, makers_FQ, of_FQ, spawn_FQ, have_FQ, ...",neg
1,"[if_FQ, you're_FQ, into_FQ, watching_FQ, near_...",neg
2,"[say_FQ, ,_FQ, tell_FQ, me_FQ, if_FQ, you've_F...",neg
3,"[the_FQ, plot_FQ, of_FQ, big_FQ, momma's_FQ, h...",neg
4,"[my_FQ, inner_FQ, flag_FQ, was_FQ, at_FQ, half...",neg


In [56]:
uni_pos_vocab, uni_pos_counts = build_vocab(uni_pos_data.text)
uni_pos_pres_data = process(uni_pos_data.text, uni_pos_vocab)
uni_pos_pres_data.head()

,the_MH,:_MH,dead_MH,but_LQ,;_LQ,then_MH,so_LQ,not_MH,NOT_the_FQ,angels_LQ,NOT_their_FQ,played_MH,NOT_a_MH,become_MH,often_FQ,grating_MH,granted_MH,more_FQ,guys_LQ,batman_FQ,he_MH,on_FQ,pictures_FQ,behold_MH,named_FQ,celluloid_LQ,book_LQ,NOT_in_LQ,(_LQ,NOT_be_LQ,are_MH,NOT_together_MH,it_LQ,realizes_MH,summer_MH,assassin_MH,virtual_MH,leader_MH,last_LQ,is_MH,...,motta_FQ,bull_FQ,motta_MH,patti's_MH,vivian_MH,valid_MH,weaknesses_MH,boone_MH,whale's_FQ,santa_MH,domingo_MH,reza_FQ,leila_MH,leila_FQ,reza_MH,leila_LQ,1962_FQ,beacham_MH,matilda_LQ,matilda_MH,protect_LQ,redefines_MH,osnard_MH,beatrice_LQ,lauri_MH,strawberry_FQ,worrall_MH,caraboo_MH,flynt's_FQ,nesmith_MH,cams_MH,corleone_MH,corleone_FQ,ness_MH,stephane_MH,stephane_FQ,hauer_MH,rohmer's_LQ,scarlett_LQ,rhett_LQ
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,False,False,False,True,True,False,True,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [0]:
uni_freq_data.to_pickle("processed_data/unigram_frequency")
uni_pres_data.to_pickle("processed_data/unigram_pres")
uni_bi_data.to_pickle("processed_data/uni_bi_pres")
bi_pres_data.to_pickle("processed_data/bi_pres")
POS_pres_data.to_pickle("processed_data/POS_pres")
adj_pres_data.to_pickle("processed_data/adj_pres")
top_uni_pres_data.to_pickle("processed_data/top_uni_pres")
uni_pos_pres_data.to_pickle("processed_data/uni_pos_pres")
sample_data['class'].to_pickle("processed_data/classes")